In [1]:
import os

In [2]:
%pwd

'd:\\mlops\\datasciencProject\\research'

In [3]:
os.chdir("d:/mlops/datasciencProject/")
%pwd

'd:\\mlops\\datasciencProject'

In [4]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class DataIngestionconfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [20]:
from src.datasciencProject.constants import *
from src.datasciencProject.utils.common import read_yaml,create_directories

In [6]:
class ConfigurationManager:
    def __init__(self,
                   config_filepath=CONFIG_FILE_PATH,
                   params_filepath=PARAMS_FILE_PATH,
                   schema_filepath=SCHEAMA_FILE_PATH):
        self.config = read_yaml(config_filepath) 
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionconfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionconfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )

        return data_ingestion_config    

In [17]:
import os
import urllib.request as request
from src.datasciencProject.utils import logger
import zipfile

In [18]:
class DataIngestion:
    def __init__(self, config: DataIngestionconfig):
        self.config = config

    def download_file(self):
         if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
         else:
            logger.info(f"File already exists")
        # Logic to download data from self.config.source_URL
         pass

    def extract_zip_file(self):
        # Logic to unzip data to self.config.unzip_dir
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [19]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-10-29 12:04:15,099 : datasciencelogger : INFO : yaml file: config\config.yaml loaded successfully]
[2025-10-29 12:04:15,101 : datasciencelogger : INFO : yaml file: params.yaml loaded successfully]
[2025-10-29 12:04:15,107 : datasciencelogger : INFO : yaml file: schema.yaml loaded successfully]
[2025-10-29 12:04:15,109 : datasciencelogger : INFO : created directory at: artifacts]
[2025-10-29 12:04:15,111 : datasciencelogger : INFO : created directory at: artifacts/data_ingestion]
[2025-10-29 12:04:15,116 : datasciencelogger : INFO : File already exists]
